In [1]:
import pandas as pd
import numpy as np
import ppscore as pps
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import os.path
from scipy import spatial
import gensim.downloader as api
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors
nltk.data.path = ['./nltk_data/']
nltk.download('vader_lexicon',nltk.data.path[0])
nltk.download('punkt',nltk.data.path[0])
nltk.download('stopwords',nltk.data.path[0])
nltk.download('averaged_perceptron_tagger',nltk.data.path[0])
nltk.download('wordnet',nltk.data.path[0])
import zipfile
with zipfile.ZipFile('./nltk_data/corpora/wordnet.zip', 'r') as zip_ref:
    zip_ref.extractall('./nltk_data/corpora/')


[nltk_data] Downloading package vader_lexicon to ./nltk_data/...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to ./nltk_data/...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to ./nltk_data/...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     ./nltk_data/...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to ./nltk_data/...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
encoding = "cp1252"
sent_data_file_name = 'beer_sent.csv'
data= pd.read_csv('BeerDataScienceProject.csv', encoding=encoding)
stringcols = data.select_dtypes(include='object').columns
data[stringcols] = data[stringcols].fillna('').astype(str)
data.dtypes

beer_ABV              float64
beer_beerId             int64
beer_brewerId           int64
beer_name              object
beer_style             object
review_appearance     float64
review_palette        float64
review_overall        float64
review_taste          float64
review_profileName     object
review_aroma          float64
review_text            object
review_time             int64
dtype: object

In [3]:
data.head(15)

,beer_ABV,beer_beerId,beer_brewerId,beer_name,beer_style,review_appearance,review_palette,review_overall,review_taste,review_profileName,review_aroma,review_text,review_time
0,5.0,47986,10325,Sausa Weizen,Hefeweizen,2.5,2.0,1.5,1.5,stcules,1.5,A lot of foam. But a lot. In the smell some ba...,1234817823
1,6.2,48213,10325,Red Moon,English Strong Ale,3.0,2.5,3.0,3.0,stcules,3.0,"Dark red color, light beige foam, average. In ...",1235915097
2,6.5,48215,10325,Black Horse Black Beer,Foreign / Export Stout,3.0,2.5,3.0,3.0,stcules,3.0,"Almost totally black. Beige foam, quite compac...",1235916604
3,5.0,47969,10325,Sausa Pils,German Pilsener,3.5,3.0,3.0,2.5,stcules,3.0,"Golden yellow color. White, compact foam, quit...",1234725145
4,7.7,64883,1075,Cauldron DIPA,American Double / Imperial IPA,4.0,4.5,4.0,4.0,johnmichaelsen,4.5,"According to the website, the style for the Ca...",1293735206
5,4.7,52159,1075,Caldera Ginger Beer,Herbed / Spiced Beer,3.5,3.5,3.0,3.0,oline73,3.5,Poured from the bottle into a Chimay goblet. A...,1325524659
6,4.7,52159,1075,Caldera Ginger Beer,Herbed / Spiced Beer,3.5,3.5,3.5,4.0,Reidrover,4.0,"22 oz bottle from ""Lifesource"" Salem. $3.95 Ni...",1318991115
7,4.7,52159,1075,Caldera Ginger Beer,Herbed / Spiced Beer,3.5,2.5,3.0,2.0,alpinebryant,3.5,"Bottle says ""Malt beverage brewed with Ginger ...",1306276018
8,4.7,52159,1075,Caldera Ginger Beer,Herbed / Spiced Beer,3.5,3.0,4.0,3.5,LordAdmNelson,4.0,I'm not sure why I picked this up... I like gi...,1290454503
9,4.7,52159,1075,Caldera Ginger Beer,Herbed / Spiced Beer,5.0,3.5,4.5,4.0,augustgarage,4.0,Poured from a 22oz bomber into my Drie Fontein...,1285632924


Q1: Rank top 3 breweries which produce the strongest beers ?

In [20]:
abv_filt = data[data['beer_ABV'].notna()]
abv_filt = abv_filt[abv_filt['beer_brewerId'].notna()]
abv_filt = abv_filt[['beer_brewerId','beer_beerId','beer_ABV']].drop_duplicates()
n_larg = 15
top_n = 5
abv_filt.groupby('beer_brewerId')['beer_ABV'].apply(lambda grp: grp.nlargest(n_larg).mean()).sort_values(ascending=False)[:top_n]

beer_brewerId
6513     24.690000
16866    17.000000
35       15.480000
736      13.500000
24215    12.466667
Name: beer_ABV, dtype: float64

So brewer_ids 6513, 16866, 35 rank top 3 in that order that produce the strongest beers according to average ABV value of the 15 strongest beers of each brewer

Q2: Which year did beers enjoy the highest ratings?

In [5]:
# Since the data looks like unix timestamp
data['review_time'] = pd.to_datetime(data['review_time'],unit='s')
data['review_year'] = pd.DatetimeIndex(data['review_time']).year
data.head(5)

,beer_ABV,beer_beerId,beer_brewerId,beer_name,beer_style,review_appearance,review_palette,review_overall,review_taste,review_profileName,review_aroma,review_text,review_time,review_year
0,5.0,47986,10325,Sausa Weizen,Hefeweizen,2.5,2.0,1.5,1.5,stcules,1.5,A lot of foam. But a lot. In the smell some ba...,2009-02-16 20:57:03,2009
1,6.2,48213,10325,Red Moon,English Strong Ale,3.0,2.5,3.0,3.0,stcules,3.0,"Dark red color, light beige foam, average. In ...",2009-03-01 13:44:57,2009
2,6.5,48215,10325,Black Horse Black Beer,Foreign / Export Stout,3.0,2.5,3.0,3.0,stcules,3.0,"Almost totally black. Beige foam, quite compac...",2009-03-01 14:10:04,2009
3,5.0,47969,10325,Sausa Pils,German Pilsener,3.5,3.0,3.0,2.5,stcules,3.0,"Golden yellow color. White, compact foam, quit...",2009-02-15 19:12:25,2009
4,7.7,64883,1075,Cauldron DIPA,American Double / Imperial IPA,4.0,4.5,4.0,4.0,johnmichaelsen,4.5,"According to the website, the style for the Ca...",2010-12-30 18:53:26,2010


In [6]:
top_n = 3
data.groupby('review_year')['review_overall'].mean().sort_values(ascending=False)[:top_n]

review_year
2000    4.181818
1999    4.000000
2001    3.927741
Name: review_overall, dtype: float64

Year 2000 had the higest average overall beer rating

Q3: Based on the user’s ratings which factors are important among taste, aroma, appearance, and palette?

In [7]:
col_list = ['review_appearance','review_palette','review_taste','review_aroma','review_overall']
f = pps.predictors(data[col_list],col_list[-1]).set_index('x')['ppscore']
f

x
review_aroma         0.353347
review_taste         0.248420
review_palette       0.161740
review_appearance    0.069060
Name: ppscore, dtype: float64

From the univariate analysis using PPS we got the decreasing order of importance according to the data is Aroma, Taste, Palette and Appearance.

Q4: If you were to recommend 3 beers to your friends based on this data which ones will you recommend?

In [8]:
# Taken from https://nbviewer.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter4_TheGreatestTheoremNeverTold/Ch4_LawOfLargeNumbers_PyMC3.ipynb#Example:-How-to-order-Reddit-submissions
def bayesian_rank(n,s):
    n = float(n)
    s = float(s)
    a = 1. + s
    b = 1. + n - s
    mu = a/(a+b)
    std_err = 1.65*np.sqrt((a*b)/(((a+b)**2)*(a+b+1.0)))
    return mu - std_err

In [9]:
rate_filt = data[data['review_overall'].notna()]
rate_filt = rate_filt[rate_filt['beer_beerId'].notna()]
top_n = 10
max_rate = 5.0
agg_dic = {'review_profileName':'count','review_overall':'sum'}
agg_df = rate_filt.groupby('beer_beerId').agg(agg_dic)
agg_df.columns = ['N','S']
agg_df = agg_df.reset_index()
agg_df['S'] = agg_df.apply(lambda row:row['S']/max_rate,axis=1)
agg_df['bayesian_rank'] = agg_df.apply(lambda row:bayesian_rank(row['N'],row['S']),axis=1)
final_df  = agg_df.sort_values('bayesian_rank',ascending=False)[:top_n]
final_df = final_df.join(data[['beer_beerId','beer_name']].drop_duplicates().set_index('beer_beerId'), on='beer_beerId')
final_df

,beer_beerId,N,S,bayesian_rank,beer_name
4279,16814,469,433.9,0.903150,Heady Topper
11986,47658,637,584.9,0.898898,Founders CBS Imperial Stout
14236,56082,252,233.4,0.895261,Citra DIPA
8983,36316,156,144.4,0.884805,Cantillon Blåbær Lambik
1667,6368,662,594.3,0.877049,Masala Mama India Pale Ale
2282,8626,41,39.1,0.870176,Southampton Berliner Weisse
5060,19960,1932,1699.2,0.866883,Founders KBS (Kentucky Breakfast Stout)
4035,15881,1955,1718.7,0.866571,Tröegs Nugget Nectar
3003,11757,2502,2179.0,0.859542,Founders Breakfast Stout
157,645,2170,1883.3,0.855543,Trappistes Rochefort 10


Formulated a bayesian analysis rank that accounts for both the number of ratings and also the value of ratings. So, from the sorting based on this rank I would recommend Heady Topper, Founders CBS Imperial Stout and Citra DIPA as the top 3.

Q5: Which Beer style seems to be the favorite based on reviews written by users? 

In [10]:
if os.path.exists(sent_data_file_name):
    rev_data = pd.read_csv(sent_data_file_name)
else:
    # Takes 8.5 mins to run because of over 500k reviews
    rev_filt = data[data['review_text']!=""]
    rev_filt = rev_filt[rev_filt['beer_style'] != ""]
    rev_data = rev_filt[['beer_style','review_text','review_overall']]

    sid = SentimentIntensityAnalyzer()
    rev_data["review_text"] = rev_data["review_text"].apply(lambda x: x.lower())
    rev_data["sentiments"] = rev_data["review_text"].apply(lambda x: sid.polarity_scores(x))
    rev_data = pd.concat([rev_data.drop(['sentiments'], axis=1), rev_data['sentiments'].apply(pd.Series)], axis=1)
    rev_data.to_csv('beer_sent.csv',index=False)

top_n = 20
agg_dic1 = {'review_text':'count','compound':'sum'}
rev_data['compound'] = rev_data['compound'].apply(lambda x: (x+1.0)/2.0)
agg_df1 = rev_data.groupby('beer_style').agg(agg_dic1)
agg_df1.columns = ['N','S']
agg_df1 = agg_df1.reset_index()
agg_df1['bayesian_rank'] = agg_df1.apply(lambda row:bayesian_rank(row['N'],row['S']),axis=1)
agg_df1.sort_values('bayesian_rank',ascending=False)[:top_n]

,beer_style,N,S,bayesian_rank
86,Quadrupel (Quad),4933,4582.27060,0.922685
11,American Double / Imperial Stout,23352,21582.97050,0.921351
58,Flanders Red Ale,2856,2642.82700,0.916937
38,Dortmunder / Export Lager,1809,1674.03310,0.914707
25,Belgian Strong Dark Ale,15403,14122.05610,0.913112
90,Rye Beer,5179,4739.14050,0.908513
84,Old Ale,4817,4408.55015,0.908408
101,Wheatwine,891,821.96225,0.906734
87,Rauchbier,2607,2385.85675,0.905841
74,Lambic - Fruit,3768,3437.93710,0.904580


I have used the very commonly used VADER sentiment analyser and used to sort by bayesian rank (from previous sections) on the compound score obtained from VADER. We get that Quadrupel and Anerican Double/Imperial Stout are the most favourite beer styles based on just written reviews.

Q6: How does written review compare to overall review score for the beer styles?

In [11]:
rev_filt = data[data['review_text']!=""]
rev_filt = rev_filt[rev_filt['beer_style'] != ""]
rev_filt = rev_filt[rev_filt['review_overall'].notna()]
top_n = 20
max_rate = 5.0
agg_dic2 = {'review_text':'count','review_overall':'sum'}
agg_df2 = rev_filt.groupby('beer_style').agg(agg_dic2)
agg_df2.columns = ['N','S']
agg_df2 = agg_df2.reset_index()
agg_df2['S'] = agg_df2.apply(lambda row:row['S']/max_rate,axis=1)
agg_df2['bayesian_rank'] = agg_df2.apply(lambda row:bayesian_rank(row['N'],row['S']),axis=1)
agg_df2.sort_values('bayesian_rank',ascending=False)[:top_n]

,beer_style,N,S,bayesian_rank
11,American Double / Imperial Stout,23352,19150.7,0.815913
63,Gueuze,1575,1304.4,0.812091
83,Oatmeal Stout,6720,5484.6,0.808270
27,Berliner Weissbier,933,771.4,0.805655
90,Rye Beer,5179,4215.7,0.804957
86,Quadrupel (Quad),4933,3994.9,0.800487
89,Russian Imperial Stout,17183,13834.7,0.800118
25,Belgian Strong Dark Ale,15403,12351.2,0.796532
38,Dortmunder / Export Lager,1809,1466.0,0.794845
12,American IPA,43364,34602.0,0.794748


We see that the top 20 of the ranking lists based on just review text and just overall ratings are very similar with almost the same beer styles juggled in positions. So, we can say that there is a good comparision between written reviews and overall ratings but this can get better with the use of more advanced sentiment analysis methods (ML or hybrid methods).

Q7: How do find similar beer drinkers by using written reviews only?

In [12]:
api.BASE_DIR = './gensim-data/'
model_name = "glove-twitter-25"
model_path = api.load(model_name,return_path=True)
model = KeyedVectors.load_word2vec_format(model_path)

In [13]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wn.ADJ
    elif pos_tag.startswith('V'):
        return wn.VERB
    elif pos_tag.startswith('N'):
        return wn.NOUN
    elif pos_tag.startswith('R'):
        return wn.ADV
    else:
        return wn.NOUN

def preprocess(text):
    text = text.lower()
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    text = [word for word in text if not any(c.isdigit() for c in word)]
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    text = [t for t in text if len(t) > 0]
    pos_tags = pos_tag(text)
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    text = [t for t in text if len(t) > 1]
    return(text)

def get_vector(s):
    return np.sum(np.array([model[i] for i in preprocess(s) if i in model.index_to_key]), axis=0)


def tagged_to_synset(word, tag):
    wn_tag = get_wordnet_pos(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 
def sentence_similarity(sentence1, sentence2):
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]

    # Filter out None values
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
    
    score, count = 0.0, 0
    sim_list = []
    for synset in synsets1:
        
        sim_list = [synset.path_similarity(ss) for ss in synsets2]
        sim_list = [0.0 if v is None else v for v in sim_list]
        best_score = max(sim_list)
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1
 
    # Average the values
    score /= count
    return score

def symmetric_sentence_similarity(sentence1, sentence2):
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1)) / 2 

def get_sim_score(s1,s2):
    score1 = symmetric_sentence_similarity(s1,s2)
    score2 = 1 - spatial.distance.cosine(get_vector(s1), get_vector(s2))
    return (score1+score2)/2.0

In [14]:
data[['review_text','review_profileName']][:20]

,review_text,review_profileName
0,A lot of foam. But a lot. In the smell some ba...,stcules
1,"Dark red color, light beige foam, average. In ...",stcules
2,"Almost totally black. Beige foam, quite compac...",stcules
3,"Golden yellow color. White, compact foam, quit...",stcules
4,"According to the website, the style for the Ca...",johnmichaelsen
5,Poured from the bottle into a Chimay goblet. A...,oline73
6,"22 oz bottle from ""Lifesource"" Salem. $3.95 Ni...",Reidrover
7,"Bottle says ""Malt beverage brewed with Ginger ...",alpinebryant
8,I'm not sure why I picked this up... I like gi...,LordAdmNelson
9,Poured from a 22oz bomber into my Drie Fontein...,augustgarage


In [15]:
symmetric_sentence_similarity(data['review_text'][0], data['review_text'][2])

0.3409403539346141

In [16]:
get_sim_score(data['review_text'][0], data['review_text'][10])

0.6566285869651112